In [1]:
import numpy as np
import io
import nltk
import keras as k
from sklearn.metrics import f1_score,accuracy_score
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, merge, Dropout, RepeatVector, Activation, merge, Lambda, Flatten, Reshape
from keras.layers import LSTM, Bidirectional, TimeDistributed, GRU, concatenate
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras import optimizers
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
import string
import argparse
from collections import Counter
import re
import argparse
import json
import sys
# from utils copy preprocess.py and attention.py to colab local location
from utils.preprocess import splitDatasets, splitValDatasets,vectorizeValData, vectorizeData
from utils.attention import Attention

Using TensorFlow backend.
C:\Users\abdeslem\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\abdeslem\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\abdeslem\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\abdeslem\anaconda3\lib\site-packages\tensorflow\python\framewo

In [2]:
embeddings_index = {}
f = open('glove/glove.6B.100d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [3]:
with open('data/train-v2.0.json') as json_data:
    d = json.load(json_data)
trainData = d['data']

In [4]:
tContext, tQuestion, tQuestion_id, tAnswerBegin, tAnswerEnd, tAnswerText, maxLenTContext, maxLenTQuestion = splitDatasets(trainData)

In [5]:

with open('data/dev-v2.0.json') as json_data:
    d = json.load(json_data)
valData = d['data']


In [6]:
vContext, vQuestion, vQuestion_id, vAnswerBegin, vAnswerEnd, vAnswerText, vmaxLenTContext, vmaxLenTQuestion = splitDatasets(valData)

In [7]:
vocab = {}
for words in tContext + tQuestion + vContext + vQuestion:
    for word in words:
        if word not in vocab:
            vocab[word] = 1
vocab = sorted(vocab.keys())

In [8]:
vocab_size = len(vocab) + 1
print(vocab_size)
word_index = dict((c, i + 1) for i, c in enumerate(vocab))
context_maxlen = max(maxLenTContext, vmaxLenTContext)
question_maxlen = max(maxLenTQuestion, vmaxLenTQuestion)

106995


In [9]:
tX, tXq, tYBegin, tYEnd = vectorizeData(tContext, tQuestion, tAnswerBegin, tAnswerEnd, word_index, context_maxlen, question_maxlen)

In [10]:
vX, vXq, vYBegin, vYEnd = vectorizeData(vContext, vQuestion, vAnswerBegin, vAnswerEnd, word_index, context_maxlen, question_maxlen)

In [11]:
nb_words = len(word_index)
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = context_maxlen

embedding_matrix = np.zeros((nb_words+1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [12]:
question_input = Input(shape=(question_maxlen,), dtype='int32', name='question_input')
context_input = Input(shape=(context_maxlen,), dtype='int32', name='context_input')
questionEmbd = Embedding(output_dim=EMBEDDING_DIM, input_dim=vocab_size,mask_zero=True, weights=[embedding_matrix], 
                         input_length=question_maxlen, trainable=False)(question_input)
contextEmbd = Embedding(output_dim=EMBEDDING_DIM, input_dim=vocab_size,mask_zero=True, weights=[embedding_matrix], 
                input_length=context_maxlen, trainable=False)(context_input)

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
from utils.attention import Attention

In [14]:
Q = Bidirectional(LSTM(128, return_sequences=True))(questionEmbd)
D = Bidirectional(LSTM(128, return_sequences=True))(contextEmbd)

In [15]:
Q2 = Attention(question_maxlen)(Q)
D2 = Attention(context_maxlen)(D)

In [16]:
L = concatenate([D2, Q2], axis=1)

In [17]:
answerPtrBegin_output = Dense(context_maxlen, activation='softmax')(L)

In [18]:
Lmerge = concatenate([L, answerPtrBegin_output],axis = 1)
answerPtrEnd_output = Dense(context_maxlen, activation='softmax')(Lmerge)

In [19]:
model = Model(input=[context_input, question_input], output=[answerPtrBegin_output, answerPtrEnd_output])
am = optimizers.Adam(lr=0.0005)
model.compile(optimizer=am, loss='categorical_crossentropy',
              loss_weights=[.04, 0.04], metrics=['accuracy'])
model.summary()

C:\Users\abdeslem\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
context_input (InputLayer)      (None, 766)          0                                            
__________________________________________________________________________________________________
question_input (InputLayer)     (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 766, 100)     10699500    context_input[0][0]              
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 60, 100)      10699500    question_input[0][0]             
____________________________________________________________________________________________

In [ ]:
train_slice = 10000
filepath="modelweightsfinal.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model_history = model.fit([tX, tXq], [tYBegin, tYEnd],batch_size= 128, verbose=1,
                          callbacks = callbacks_list,epochs=30)

Epoch 1/30
49280/86821 [================>.............] - ETA: 2:08:46 - loss: 0.4112 - dense_1_loss: 5.1240 - dense_2_loss: 5.1569 - dense_1_accuracy: 0.0243 - dense_2_accuracy: 0.0137